# Vermeer style identification

# Training a classifier

In a previous notebook we have extracted features from the paintings by Johannes Vermeer and performed some visualization techniques of the feature vectors obtained. Here we will repeat the process for the ``no vermeer`` class, corresponding to paintings of other Dutch artists of the same period, and use the whole set of features train a classifier to distinguish between Vermeer and other artists. Concretely, we will focus on the features extracted by ResNet50, which showed a better feature extraction capability in the previous notebook. Finally we will test the model on a set of paintings that includes "Girl with a flute", the painting whose provenance was questioned.

## Preprocessing of paintings by other artists

We follow the same steps as in the previous notebook for preprocessing our set of images of paintings by other artists from the Dutch Baroque: first cropping the images into squares and the resizing to match the usual input format for CNNs. 

In [1]:
from PIL import Image
import os

def sweep_square_and_crop(folder_path, output_folder, step_size=200):
    """
    Sweep a square over each image and crop it to produce multiple square images.

    Args:
    folder_path (str): Path to the folder containing images.
    output_folder (str): Path to the folder where the squared images will be saved.
    step_size (int): The step size for sweeping the square over the image. Fixed to 200 steps by default.

    Returns:
    None
    """
    # Create the output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            with Image.open(image_path) as img:
                # Determine the size of the square (minimum of width and height of the image)
                square_size = min(img.size)

                for x in range(0, img.width - square_size + 1, step_size):
                    for y in range(0, img.height - square_size + 1, step_size):
                        # Define the crop box
                        box = (x, y, x + square_size, y + square_size)
                        cropped_image = img.crop(box)

                        # Construct the output filename
                        output_filename = f"{os.path.splitext(filename)[0]}_{x}_{y}.png"
                        # Save the cropped image
                        cropped_image.save(os.path.join(output_folder, output_filename))

In [2]:
sweep_square_and_crop('./no vermeer/details', './no vermeer/cropped', step_size=150)

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def resize(input_folder, output_folder, size=(224, 224)):
    """
    Resize images to the given size and save them as
    numpy arrays in a separate folder.

    Args:
    - input_folder (str): Folder containing the original images.
    - output_folder (str): Folder where numpy arrays of processed images will be saved.
    - size (tuple): New size of the images. Default is (224, 224).
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Resize images
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(input_folder, filename)
            img = load_img(img_path, target_size=size)
            img_array = img_to_array(img)

            # Save the numpy array
            array_filename = os.path.splitext(filename)[0] + '.npy'
            save_path = os.path.join(output_folder, array_filename)
            np.save(save_path, img_array)

In [4]:
source_folder = './no vermeer/cropped'
destination_folder = './no vermeer/preprocessed'
resize(source_folder, destination_folder)

The resized images have been stored as ``.npy`` files in te folder ``no vermeer\preprocessed``. Note that we have a rather balanced dataset (758 Vermeer versus 685 no Vermeer).

## Extracting features and creating labels

In this section we will extract ResNet50 features for each ``.npy`` image in the ``vermeer\preprocessed`` and ``no vermeer\preprocessed`` folders and creates labels for each class. Note that we will have to use the function ``preprocess_input`` in Keras in order to the image data to match the format and distribution expected by ResNet5t.  

In [1]:
import numpy as np
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Load the pre-trained ResNet50 model without the top layer (for feature extraction)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# This function loads the images in "folder"
def load_npy_images(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".npy"):
            img_path = os.path.join(folder, filename)
            img = np.load(img_path)
            images.append(img)
    return np.array(images)

# This function extracts features in the numpy array of images "images"
def extract_features(images):
    # Preparing the image data to match the format and distribution expected by ResNet50
    images = preprocess_input(images)
    # Extract features
    features = feature_extractor.predict(images, batch_size=16)
    return features

# Load images in our directories
vermeer_images = load_npy_images('./vermeer/preprocessed')
no_vermeer_images = load_npy_images('./no vermeer/preprocessed')


In [2]:
# Extract features
vermeer_features = extract_features(vermeer_images)
no_vermeer_features = extract_features(no_vermeer_images)

# Create labels (1 for Vermeer, 0 for no Vermeer)
vermeer_labels = np.ones(len(vermeer_features))
no_vermeer_labels = np.zeros(len(no_vermeer_features))

43/43 [==============================] - 67s 2s/step


We shall combine the features and labels and split the dataset into a train and validation set. The former will serve us to test the metrics of our model.

In [3]:
# Combine features and labels
features = np.concatenate((vermeer_features, no_vermeer_features), axis=0)
labels = np.concatenate((vermeer_labels, no_vermeer_labels), axis=0)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

## Classification

Now we have our correctly preprocessed train and validation sets it is time to perform classification. We will use a Support Vector Machine (SVM) as our classifier- SVMs are known for their effectiveness in dealing with high-dimensional spaces, making them suitable for being used in complex feature sets derived from artworks. 

In [4]:
# Train the SVM classifier
svm_clf = make_pipeline(StandardScaler(), SVC(probability=True))
svm_clf.fit(X_train, y_train)

# Predict labels for the validation set
y_pred = svm_clf.predict(X_val)

Note we are using SVC with ``probability=True``: the SVM classifier is equipped to output probabilities for each class. These probabilities are derived using an additional logistic regression on the decision values returned by the SVM, fitted via a process called Platt scaling during the fit method. The classification into 0 or 1 happens based on these probabilities: the class with the highest probability is chosen as the prediction for each sample. In our case, if the probability of the positive class is greater than or equal to 0.5, the sample is classified as 1. Otherwise, it is classified as 0.

We shall now evaluate the model in our validation set:

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# For ROC AUC score, we need the probability estimates of the positive class
y_proba = svm_clf.predict_proba(X_val)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_proba)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
svm_clf = make_pipeline(StandardScaler(), SVC(probability=True))
svm_clf.fit(X_train, y_train)


Accuracy: 0.9689
Precision: 0.9618
Recall: 0.9805
F1 Score: 0.9711
ROC AUC Score: 0.9971


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True))])

The results indicate that our model performs quite well on the validation set. Let us recall the meaning of these metrics:

Accuracy (0.9689): the classifier correctly predicts the class of 96.89% of the samples, indicating overall good performance across both classes.

Precision (0.9618): when the classifier predicts a sample as belonging to the positive class, it is correct 96.18% of the time. High precision indicates a low false positive rate.

Recall (0.9805): The classifier correctly identifies 98.05% of all actual positive samples. High recall indicates a low false negative rate.

F1 Score (0.9711): The F1 score is the harmonic mean of precision and recall, providing a single metric to measure the balance between them. The F1 score close to 1, suggesting that the classifier achieves an excellent balance between precision and recall.

ROC AUC Score (0.9971): the Area Under the Receiver Operating Characteristic Curve (ROC AUC) measures the classifier's ability to distinguish between the classes across all possible thresholds. A score close to 1 indicates an excellent ability to differentiate between positive and negative classes. This suggests that the classifier is highly reliable in its predictions.

As a note, the precision being slightly lower than recall (but still high) suggests that there are some false positives, but their number is very low compared to true positives.

## Predictions on a test set

We shall now address the original question: what does the model say about the provenance of "Girl with a flute"?

We shall consider a test set of paintings and some patches thereof, including "Girl with a flute". For Vermeer paintings, we will use pictures obtained from a different source than the one used for training the model. For paintings by other artists, we will simply use other artworks not used in our previous dataset. The images have been stored in the folder ``./test``. 

As usual, we need to correctly process these images. The processed pictures have been stored in ``./test/preprocessed``. 

In [ ]:
source_folder = './test/cropped'
destination_folder = './test/preprocessed'
resize(source_folder, destination_folder)

We now create a function that extracts features from the images in our test folder and outputs the probability of being a Vermeer computed by our model:

In [20]:
def predict_vermeer_probabilities_svm(image_folder, svm_clf):
    """Predict the probability of .npy images being Vermeer paintings using the trained SVM classifier."""
    # Load .npy images from the specified folder
    images = load_npy_images(image_folder)  # Reuse the function defined earlier
    
    # Extract features using the pre-trained ResNet
    features = extract_features(images)  # Reuse the function defined earlier
    
    # Predict probabilities using the trained SVM classifier
    probabilities = svm_clf.predict_proba(features)[:, 1]  # Get probability for the positive class (Vermeer)
    
    # Print the predicted probabilities for each image
    for idx, prob in enumerate(probabilities):
        print(f"Image {idx+1}: Predicted probability of being a Vermeer painting: {prob}")

# Specify the folder containing .npy test images
test_image_folder = './test/preprocessed'

# Predict and print the probabilities
predict_vermeer_probabilities_svm(test_image_folder, svm_clf)

3/3 [==============================] - 5s 1s/step
Image 1: Predicted probability of being a Vermeer painting: 0.025572119523263524
Image 2: Predicted probability of being a Vermeer painting: 0.07385237094306933
Image 3: Predicted probability of being a Vermeer painting: 0.09684409436667323
Image 4: Predicted probability of being a Vermeer painting: 0.0019061251631808326
Image 5: Predicted probability of being a Vermeer painting: 0.010464657595303363
Image 6: Predicted probability of being a Vermeer painting: 0.9961084864787438
Image 7: Predicted probability of being a Vermeer painting: 0.986035101824995
Image 8: Predicted probability of being a Vermeer painting: 0.9204058822877744
Image 9: Predicted probability of being a Vermeer painting: 0.4509658444854587
Image 10: Predicted probability of being a Vermeer painting: 0.9291889975668969
Image 11: Predicted probability of being a Vermeer painting: 0.5197314690690261
Image 12: Predicted probability of being a Vermeer painting: 0.98610435

Let us analyze the meaning of these results. The first five paintings correspond to paintings by other artists, so the classifier is doing its job well. Image 6 is "Girl with a flute", and Image 7 to Image 11 are different patches of the painting.  While the whole picture is classified as a Vermeer, the output probability of the different zoomed-in patches differ. Crucially, for patches involving the face, hand and central part of the jacket of the portrayed character, the classifier outputs a high probability of being a Vermeer, whereas for peripheric details the probability is considerably lower. 

Details with high probability of belonging to Vermeer class:

<img src="./test/cropped/flute1.png" width="400" height="300" alt="description">

<img src="./test/cropped/flute2.png" width="400" height="300" alt="description">

<img src="./test/cropped/flute6.png" width="400" height="300" alt="description">

<img src="./test/cropped/flute8.png" width="400" height="300" alt="description">

Details with low probability of belonging to Vermer class:

<img src="./test/cropped/flute5.png" width="400" height="300" alt="description">

<img src="./test/cropped/flute7.png" width="400" height="300" alt="description">

<img src="./test/cropped/flute9.png" width="400" height="300" alt="description">

Images 14, 16 and 20 are forgeries from the 20th century, in the style of Vermeer. Images 15, 17-19 and 21-22 are their respective zoomed-in details. While for the whole images the classifier outputs a high probability, zoomed-in patches result in a low probability of being a Vermeer. This suggests that our model recognizes a similar style in forgeries as a whole but is able to tell them apart from originals when focusing on details. 

<img src="./test/cropped/forgery1.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery1-1.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery2.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery2-1.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery2-2.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery3.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery3-1.png" width="400" height="300" alt="description">

<img src="./test/cropped/forgery3-3.png" width="400" height="300" alt="description">

Finally, the rest of the pictures are all by Vermeer, either complete pictures or details. We observe that the model outputs a very high probability for all of them. Recall that these pictures have been taken from a different source, so they are in practice different instances from those in the original dataset used for training and validation. 

## Conclusion

Our results suggests that our model:
    
1. indeed tells apart paintings by Vermeer from other painters of the same school but with different style, 
2. for imitators, the whole painting passes the test but zommed-in details do not.

The situation is not so clear for "Girl with a flute": while the whole painting is indeed classified as a Vermeer, several zoomed-in patches are not. The fact that the positive patches are the most relevant of the artwork (face, hand, central piece of the character's jacket) while the negative ones correspon to peripheric details (hat, wristle details of the jacket, chair in the background) strengthen the hypothesis that the painting belongs to Vermeer's studio, with Vermeer in charge of the main aspects of the portrait and some pupil taking care of the rest of the painting. 

It would be interesting to perform further studies that either refine our current model or consider a different model for comparison, and also refining our feature extraction scheme. For instance, we can try a dense layer as a classifier instead of a SVM. Adding to our current model the texture and edge detection features that were described in the previous notebook does not produce significant changes in the output, but we could certainly consider  extracting features with a less sophisticated custom CNN and complement them with texture and edge information, and compare the performance of classifiers. Other nuances in the feature extraction process include taking care of lighting: given the fact that the Vermeer paintings (including "Girl with a flute" were taken under similar lightning conditions for the 2023 exhibition at Reijksmuseum, we could introduce brightness and contrast variations in that set to prevent bias to particular lightning conditions. Finally, we can also improve the dataset we have used by further data augmentation, taking a larger number of well-selected zoomed-in patches.  